In [ ]:
!pip install lxml pulsar-client

In [ ]:
from bs4 import BeautifulSoup
import requests
import json

# Testing sending messages to Pulsar--this task should be performed by the scraper
oai_uri = "http://export.arxiv.org/oai2?set=cs&verb=ListIdentifiers&metadataPrefix=oai_dc&from=2025-03-08"
page = requests.get(oai_uri)
oai_soup = BeautifulSoup(page.content, "xml")
ids = oai_soup.find_all("identifier")
urls = [id.get_text().replace('oai:arXiv.org:','https://arxiv.org/abs/') for id in ids][:10]

def process_resource(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    title = soup.find("h1",class_='title mathjax').get_text().replace('Title:','')
    summary = soup.find("blockquote", class_='abstract mathjax').get_text().replace('Abstract:','').strip()
    authors = '\n'.join([a.get_text() + " (" + a.get('href') + ")" for a in soup.find("div",class_='authors').find_all('a')])
    pdf_url = 'https://arxiv.org' + soup.find('a',class_='download-pdf').get('href')
    return json.dumps({"title": title, "summary": summary, "authors": authors, "url": pdf_url})

msgs = [process_resource(url) for url in urls]
print(len(msgs))
print(msgs[0])

In [ ]:
import pulsar
procli = pulsar.Client('pulsar://pdf-service:6650')
producer = procli.create_producer('scraper-output')

for msg in msgs:
    producer.send(msg.encode('utf-8'))

producer.close()